# Starbucks Stores Analysis

In [1]:
# Housekeeping
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import math

## Datasets

Data Constraints:
- Both Starbucks and US datasets published in 2017.
- Starbucks store locations limited to US country. 
- Starbucks store limited to Starbucks brand (no Teavana)
- Exclude Puerto Rico from US datasets

In [2]:
starbucks = pd.read_csv('data/directory.csv')
starbucks = starbucks.query("Brand == 'Starbucks'").query("Country == 'US'")
starbucks = starbucks.drop(columns=["Brand", "Store Name", "Ownership Type", "Street Address","Phone Number","Timezone", "Postcode", "Country"])
starbucks = starbucks.rename(columns={'State/Province' : 'State'})

In [3]:
cities = pd.read_csv('data/uscities.csv')
cities = cities[["city", "state_id", "state_name", "county_name"]]

In [4]:
demographic = pd.read_csv('data/demo.csv', encoding='cp1252')
demographic = demographic[demographic['State'] != 'Puerto Rico']
demographic["County"] = demographic["County"].apply(lambda x: ' '.join(x.split()[0:-1]))

### Data Clean Up

In [5]:
mapping = pd.merge(starbucks, cities, left_on=["City", "State"], right_on=["city", "state_id"])
mapping = mapping.drop(columns=["state_id", "city", "State"])
mapping = mapping.rename(columns={"state_name":"State", "county_name":"County"})
mapping

,Store Number,City,Longitude,Latitude,State,County
0,3513-125945,Anchorage,-149.78,61.21,Alaska,Anchorage
1,74352-84449,Anchorage,-149.84,61.14,Alaska,Anchorage
2,12449-152385,Anchorage,-149.85,61.11,Alaska,Anchorage
3,24936-233524,Anchorage,-149.89,61.13,Alaska,Anchorage
4,8973-85630,Anchorage,-149.86,61.14,Alaska,Anchorage
...,...,...,...,...,...,...
12119,22353-220004,Lander,-108.75,42.84,Wyoming,Fremont
12120,74385-87621,Laramie,-105.59,41.32,Wyoming,Albany
12121,73320-24375,Laramie,-105.56,41.31,Wyoming,Albany
12122,22425-219024,Laramie,-105.56,41.31,Wyoming,Albany


In [6]:
storecount = mapping.groupby(['County', 'State'])['Store Number'].count().to_frame().reset_index()
storecount = storecount.rename(columns={"Store Number":"Count"})
storecount

,County,State,Count
0,Ada,Idaho,32
1,Adair,Missouri,1
2,Adams,Colorado,62
3,Adams,Illinois,2
4,Adams,Pennsylvania,1
...,...,...,...
1026,York,Pennsylvania,10
1027,York,South Carolina,8
1028,York,Virginia,2
1029,Yuba,California,1


In [7]:
df = storecount.merge(demographic, how='right', left_on=['County', 'State'], right_on=['County', 'State']).drop(columns=["Unnamed: 0", "CountyId", "VotingAgeCitizen"])
df['Count'] = df['Count'].fillna(0)
df['Men'] = (df['Men']/df['TotalPop'])*100
df['Women'] = (df['Women']/df['TotalPop'])*100
df['Employed'] = (df['Employed']/df['TotalPop'])*100
df

,County,State,Count,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,Autauga,Alabama,2.0,55036,48.875282,51.124718,2.7,75.4,18.9,0.3,...,0.6,1.3,2.5,25.8,43.811323,74.1,20.2,5.6,0.1,5.2
1,Baldwin,Alabama,5.0,203360,48.941286,51.058714,4.4,83.1,9.5,0.8,...,0.8,1.1,5.6,27.0,44.023899,80.7,12.9,6.3,0.1,5.5
2,Barbour,Alabama,0.0,26201,53.341476,46.658524,4.2,45.7,47.8,0.2,...,2.2,1.7,1.3,23.4,33.884203,74.1,19.1,6.5,0.3,12.4
3,Bibb,Alabama,0.0,22580,54.255979,45.744021,2.4,74.6,22.0,0.4,...,0.3,1.7,1.5,30.0,36.186891,76.0,17.4,6.3,0.3,8.2
4,Blount,Alabama,0.0,57667,49.404339,50.595661,9.0,87.4,1.5,0.3,...,0.4,0.4,2.1,35.0,37.074930,83.9,11.9,4.0,0.1,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,Sweetwater,Wyoming,1.0,44527,51.611382,48.388618,16.0,79.6,0.8,0.6,...,2.8,1.3,1.5,20.5,51.067891,78.4,17.8,3.8,0.0,5.2
3138,Teton,Wyoming,2.0,22923,53.086420,46.913580,15.0,81.5,0.5,0.3,...,11.7,3.8,5.7,14.3,63.220346,82.1,11.4,6.5,0.0,1.3
3139,Uinta,Wyoming,0.0,20758,51.030928,48.969072,9.1,87.7,0.1,0.9,...,1.1,1.3,2.0,19.9,45.900376,71.5,21.5,6.6,0.4,6.4
3140,Washakie,Wyoming,0.0,8253,49.897007,50.102993,14.2,82.2,0.3,0.4,...,6.9,1.3,4.4,14.3,46.443717,69.8,22.0,8.1,0.2,6.1


### Correlation

In [8]:
var = ['Count', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'Income', 'IncomeErr',
       'IncomePerCap', 'IncomePerCapErr', 'Poverty', 'ChildPoverty',
       'Professional', 'Service', 'Office', 'Construction', 'Production',
       'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp', 'WorkAtHome',
       'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork', 'SelfEmployed',
       'FamilyWork', 'Unemployment']
corr = df[var].corr().drop('Count')[['Count']]
best_corr = corr[abs(corr["Count"])>.19]
best_corr

,Count
TotalPop,0.896795
White,-0.202117
Asian,0.450766
Income,0.233709
IncomePerCap,0.256303
Professional,0.247451
Construction,-0.212778
Transit,0.327334


### Preprocessing

In [9]:
feature_names = ['TotalPop', 'Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific', 'IncomePerCap', 'Poverty',\
                 'Professional', 'Service', 'Office', 'Construction', 'Production', 'Drive', 'Carpool', 'Transit',\
                 'Walk', 'OtherTransp']
features = df[var[1:]].fillna(0)
features = features.apply(lambda x: stats.zscore(x))
target = df[["Count"]]
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

In [ ]:
x_train_temp, x_test_temp, y_train_temp, y_test_temp = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

for i in range(1,5):
    poly_model = make_pipeline(PolynomialFeatures(i), LinearRegression())
    d = pd.DataFrame(cross_validate(poly_model, x_train_temp, y_train_temp, scoring=('r2', 'neg_mean_squared_error')))
    print(d.mean())
    

fit_time                        0.013379
score_time                      0.007930
test_r2                         0.762336
test_neg_mean_squared_error   -78.182999
dtype: float64
fit_time                         0.087059
score_time                       0.007910
test_r2                         -2.985972
test_neg_mean_squared_error   -998.747508
dtype: float64
fit_time                          1.604663
score_time                        0.023843
test_r2                         -47.148862
test_neg_mean_squared_error   -9892.625115
dtype: float64


### Linear Regression

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(r2_score(y_test, y_pred))

In [ ]:
# from tensorflow.keras import models, layers

# model = models.Sequential()
# model.add(layers.Dense(128, activation='relu', input_shape=(len(feature_names),)))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(1, activation='linear'))
# model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
# model.fit(x_train, y_train, epochs = 100, batch_size=32)
# results = model.evaluate(x_test, y_test)
# results

## Data Visualization

In [ ]:
def normalize(df, columns):
    result = df[columns]
    result = (result - result.mean())/result.std()
    return result

data = df[["TotalPop","White","Non White","IncomePerCap","Professional","Construction","Transit"]]
columns = ["TotalPop","White","Non White","IncomePerCap","Professional","Construction","Transit"]
data = normalize(data, columns)
data = data.join(df['Count'])

# Feature = TotalPop
data.plot.scatter(x="TotalPop", y="Count",s=1)
plt.ylabel("# of Starbucks")
plt.xlabel("Total Population")
plt.title("Number of Starbucks by Feature")

# Feature = White Population
data.plot.scatter(x="White", y="Count",s=1)
plt.ylabel("# of Starbucks")
plt.xlabel("Proportiona of Population that is White")
plt.title("Number of Starbucks by Feature")

# Feature = Non White Population
data.plot.scatter(x="Non White", y="Count",s=1)
plt.ylabel("# of Starbucks")
plt.xlabel("Proportion of Population that is Non White")
plt.title("Number of Starbucks by Feature")

# Feature = Income Per Capita
data.plot.scatter(x="IncomePerCap", y="Count",s=1)
plt.ylabel("# of Starbucks")
plt.xlabel("Income Per Capita")
plt.title("Number of Starbucks by Feature")

# Feature = Professional Population
data.plot.scatter(x="TotalPop", y="Count",s=1)
plt.ylabel("# of Starbucks")
plt.xlabel("Proportion of Population that are Professionals")
plt.title("Number of Starbucks by Feature")

# Feature = Construction Population
data.plot.scatter(x="Construction", y="Count",s=1)
plt.ylabel("# of Starbucks")
plt.xlabel("Proportion of Population that work in Construction")
plt.title("Number of Starbucks by Feature")

# Feature = White Population
data.plot.scatter(x="Transit", y="Count",s=1)
plt.ylabel("# of Starbucks")
plt.xlabel("Proportion of Population that work in Transit")
plt.title("Number of Starbucks by Feature")